In [133]:
import numpy as np
from PIL import Image
import image_processing as ip
import pandas as pd

In [30]:
def in_bounds(img, i, j):
    width, height = img.size
    return i >= 0 and i < width and j >= 0 and j < height

In [31]:
def get_neighbors(img, i, j, radius):
    neighbors = []
    for r in range(1, radius + 1):
        if in_bounds(img, i - r, j): # arriba
            neighbors.append((i - r, j))
        if in_bounds(img, i - r, j + r): # arriba derecha
            neighbors.append((i - r, j + r))
        if in_bounds(img, i, j + r): # derecha
            neighbors.append((i, j + r))
        if in_bounds(img, i + r, j + r): # abajo derecha
            neighbors.append((i + r, j + r))
        if in_bounds(img, i + r, j): # abajo
            neighbors.append((i + r, j))
        if in_bounds(img, i + r, j - r): # abajo a la izquierda
            neighbors.append((i + r, j - r))
        if in_bounds(img, i, j - r): # izquierda
            neighbors.append((i, j - r))
        if in_bounds(img, i - r, j - r): # arriba a la izquierda
            neighbors.append((i - r, j - r))
    return neighbors

In [32]:
def euclidean_dist(point1, point2):
    return np.linalg.norm(point1 - point2)

In [38]:
def get_density(img, i, j, t):
    neighbors = get_neighbors(img, i, j, 1)
    eps = 0
    for neighbor in neighbors:
        n_i, n_j = neighbor
        r1, g1, b1 = img.getpixel((i, j))
        r2, g2, b2 = img.getpixel((n_i, n_j))
        p1 = np.array([r1, g1, b1])
        p2 = np.array([r2, g2, b2])
        if euclidean_dist(p1, p2) <= t:
            eps += 1
    return eps

In [125]:
def dbscan(img_path, t, r, eps):
    img = Image.open(img_path).convert("RGB")
    width, height = img.size
    visited = [[False for _ in range(height)] for _ in range(width)]
    cluster = [[-1 for _ in range(height)] for _ in range(width)]
    cur_cluster = 0
    for i in range(width):
        for j in range(height):
            if visited[i][j]:
                continue
            visited[i][j] = True
            den = get_density(img, i, j, t)
            if den >= eps:
                cluster[i][j] = cur_cluster
                cur_cluster += 1
                neighbors = get_neighbors(img, i, j, r)
                while len(neighbors) > 0:
                    n_i, n_j = neighbors.pop()
                    if visited[n_i][n_j]:
                        continue
                    visited[n_i][n_j] = True
                    den = get_density(img, n_i, n_j, t)
                    if den >= eps:
                        cluster[n_i][n_j] = cluster[i][j]
                        new_neighbors = get_neighbors(img, n_i, n_j, r)
                        for new_neighbor in new_neighbors:
                            neighbors.append(new_neighbor)
            else:
                cluster[i][j] = -1
    # print(cur_cluster)
    return cluster, cur_cluster

In [115]:
class Pixel:
    def __init__(self, rgb, x, y):
        self.rgb = rgb
        self.x = x
        self.y = y

In [129]:
def paint_brain(clusters, total_clusters):
    total_colors = total_clusters
    colors = [list(np.random.choice(range(256), size=3)) for _ in range(total_colors + 1)]
    img = []
    tumor_cluster_pixels = []
    for i in range(len(clusters)):
        row = []
        for j in range(len(clusters[0])):
            if clusters[i][j] != -1:
                row.append(colors[clusters[i][j]])
            else:
                row.append([0, 0, 0])
                p = Pixel([0, 0, 0], i, j)
                tumor_cluster_pixels.append(p)
        img.append(np.array(row))
    img = np.array(img)
    return Image.fromarray(img.astype(np.uint8)), tumor_cluster_pixels

In [123]:
clusters, total_clusters = dbscan("data/dataset/107.jpg", t=8, r=15, eps=5)
_, tumor_cluster_pixels = paint_brain(clusters, total_clusters)
ip.analyze("107.jpg", tumor_cluster_pixels)

1
total clusters 1
El ratio de puntos para la imagen 107.jpg es: 20.757592249918364%


In [136]:
# experimentación
parametros_t = [0.1, 0.5, 1, 5, 10]
parametros_r = [20]
parametros_eps = [1, 3, 5, 7]

for t in parametros_t:
    for eps in parametros_eps:
        clusters, total_clusters = dbscan("data/dataset/107.jpg", t=t, r=15, eps=eps)
        _, tumor_cluster_pixels = paint_brain(clusters, total_clusters)
        # print("t", t, "r", 15, "eps", eps)

El ratio de puntos para la imagen 107.jpg es: 21.26087412220941%
El ratio de puntos para la imagen 107.jpg es: 19.980497912667214%
El ratio de puntos para la imagen 107.jpg es: 18.573793250747546%
El ratio de puntos para la imagen 107.jpg es: 16.270048320137928%
El ratio de puntos para la imagen 107.jpg es: 21.26087412220941%
El ratio de puntos para la imagen 107.jpg es: 19.980497912667214%
El ratio de puntos para la imagen 107.jpg es: 18.573793250747546%
El ratio de puntos para la imagen 107.jpg es: 16.270048320137928%
El ratio de puntos para la imagen 107.jpg es: 21.26087412220941%
El ratio de puntos para la imagen 107.jpg es: 19.980497912667214%
El ratio de puntos para la imagen 107.jpg es: 18.573793250747546%
El ratio de puntos para la imagen 107.jpg es: 16.270048320137928%
El ratio de puntos para la imagen 107.jpg es: 16.184892282519773%
El ratio de puntos para la imagen 107.jpg es: 20.69879119059447%
El ratio de puntos para la imagen 107.jpg es: 21.88390458357079%
El ratio de pun